In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import gym, rec_gym
import numpy as np
import tensorflow as tf

from rec_gym.runner import run_experiment
import gin
from rec_gym.envs import prim_env_v1, prim_env_v2

import pickle

/home/anton/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
gin.parse_config("""
PrimEnv1.n_items = 100
PrimEnv1.n_users = 10
PrimEnv1.n_rec   = 4
PrimEnv1.embedding_dimension = 2
PrimEnv1.cluster_var = 16
PrimEnv1.in_cluster_var = 4
PrimEnv1.user_change_prob = 0.05
PrimEnv1.reward_noise = 0
PrimEnv1.user_init_sigma = 4
PrimEnv1.user_ar_coef = 1
PrimEnv1.user_drift_sigma = 0
PrimEnv1.seed = 53
PrimEnv1.user_type = 'drifting'

PrimEnv2.n_items = 100
PrimEnv2.n_users = 10
PrimEnv2.n_rec   = 4
PrimEnv2.embedding_dimension = 2
PrimEnv2.cluster_var = 16
PrimEnv2.in_cluster_var = 4
PrimEnv2.user_change_prob = 0.05
PrimEnv2.reward_noise = 0
PrimEnv2.user_init_sigma = 4
PrimEnv2.user_ar_coef = 1
PrimEnv2.user_drift_sigma = 0
PrimEnv2.seed = 53
PrimEnv2.user_type = 'none'
PrimEnv2.new_items_interval = 1000
PrimEnv2.new_items_size = 0.
PrimEnv2.return_items_objects = False


PrimEnv3.n_items = 100
PrimEnv3.n_users = 10
PrimEnv3.n_rec   = 4
PrimEnv3.embedding_dimension = 2
PrimEnv3.cluster_var = 16
PrimEnv3.in_cluster_var = 4
PrimEnv3.user_change_prob = 0.05
PrimEnv3.reward_noise = 0
PrimEnv3.user_init_sigma = 4
PrimEnv3.user_ar_coef = 1
PrimEnv3.user_drift_sigma = 0
PrimEnv3.seed = 53
PrimEnv3.user_type = 'none'
PrimEnv3.new_items_interval = 1000
PrimEnv3.new_items_size = 0.
PrimEnv3.return_items_objects = False
PrimEnv3.user_graph_connectivity = 0.1
PrimEnv3.broadcast_std = 1


""")

tf.reset_default_graph()
sess = tf.InteractiveSession()

def make_env():
    env = gym.make('prim-gym-v3')
    return env

In [5]:
from agents.ddpg import DDPGAgent 

Using TensorFlow backend.


In [6]:
env = make_env()

state_dim = env.embedding_dimension
action_dim = env.embedding_dimension
action_size = env.n_rec

agent = DDPGAgent(action_size = action_size,
                 state_dim = state_dim,
                 action_dim = action_dim,
                 gamma = .9,
                 sess = sess,
                 optimizer = tf.train.AdamOptimizer(
                     learning_rate=0.001
                 ),
                 max_tf_checkpoints_to_keep = 3,
                 experience_size = 1000,
                 batch_size = 64)

In [7]:
sess.run(tf.global_variables_initializer())
run_experiment(env, agent, t_train=1000, t_test=200)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/media/anton/0b9d5a44-bb63-4de1-9cfe-0da47b81e88a/TPT/3A/PRIM Vente Privee/rec_gym/rec_gym/envs/prim_env_v3.py:190: RuntimeWarning: invalid value encountered in less
  p=[1 - p, p]
100%|██████████| 200/200 [00:07<00:00, 26.63it/s]


In [ ]:
with open('./logs/ddpg_no_random_init_env_finished.npy', 'wb') as f:
    pickle.dump(file=f, obj=env)

In [8]:
from agents.utils import data_exploring_widget
data_exploring_widget(env)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time:', max=1200), Output()), _…

In [8]:
from agents.baselines import RandomAgent

env = make_env()

state_dim = env.embedding_dimension
action_dim = env.embedding_dimension
action_size = env.n_rec

agent = RandomAgent(action_size = action_size)

In [9]:
sess.run(tf.global_variables_initializer())
run_experiment(env, agent, t_train=1000, t_test=200)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/media/anton/0b9d5a44-bb63-4de1-9cfe-0da47b81e88a/TPT/3A/PRIM Vente Privee/rec_gym/rec_gym/envs/prim_env_v3.py:190: RuntimeWarning: invalid value encountered in less
  p=[1 - p, p]
100%|██████████| 200/200 [00:00<00:00, 2063.98it/s]


In [10]:
from agents.utils import data_exploring_widget
data_exploring_widget(env)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time:', max=1200), Output()), _…

In [11]:
from agents.dqn import Qagent

env = make_env()

state_dim = env.embedding_dimension
action_dim = env.embedding_dimension
action_size = env.n_rec

agent = Qagent(sess=sess,
                 state_dim=(state_dim,),
                 action_dim=(action_dim,),
                 epsilon=0.4,
                 action_size=action_size,
                 logdir='./logs/',
                 replay_size=1000,
                 batch_size=2)

In [12]:
sess.run(tf.global_variables_initializer())
run_experiment(env, agent, t_train=1000, t_test=200)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anton/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/media/anton/0b9d5a44-bb63-4de1-9cfe-0da47b81e88a/TPT/3A/PRIM Vente Privee/rec_gym/rec_gym/envs/prim_env_v3.py:190: RuntimeWarning: invalid value encountered in less
  p=[1 - p, p]
100%|██████████| 200/200 [00:00<00:00, 307.58it/s]


In [13]:
from agents.utils import data_exploring_widget
data_exploring_widget(env)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time:', max=1200), Output()), _…